# Coronavirus distribution maps in MX 
Source: Government open coronavirus database, INEGI coordenates catalog

In [2]:
###### ------------------- Main libraries -------------------################
import pandas as pd
import numpy as np 
from pprint import pprint
import folium 
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import datetime

# Coordenates Catalog 

In [4]:
coordenadas = pd.read_csv('C:/Users/Data Analyst/Documents/Otros/COVID-19_MX/coordenadas/coordenadas.csv')
coordenadas.head()

C:\Users\Data Analyst\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Mapa,Cve_Ent,Nom_Ent,Nom_Abr,Cve_Mun,Nom_Mun,Cve_Loc,Nom_Loc,Ãmbito,Latitud,Longitud,Lat_Decimal,Lon_Decimal,Altitud,Cve_Carta,Pob_Total,Pob_Masculina,Pob_Femenina,Total De Viviendas Habitadas
0,10010001,1,Aguascalientes,Ags.,1,Aguascalientes,1,Aguascalientes,U,"21Â°52Â´47.362N""","102Â°17Â´45.768W""",21.879823,-102.296047,1878,F13D19,722250,348722,373528,185120
1,10010094,1,Aguascalientes,Ags.,1,Aguascalientes,94,Granja Adelita,R,"21Â°52Â´18.749N""","102Â°22Â´24.710W""",21.871875,-102.373531,1902,F13D18,14,*,*,2
2,10010096,1,Aguascalientes,Ags.,1,Aguascalientes,96,Agua Azul,R,"21Â°53Â´01.522N""","102Â°21Â´25.639W""",21.883756,-102.357122,1861,F13D18,37,21,16,11
3,10010100,1,Aguascalientes,Ags.,1,Aguascalientes,100,Rancho Alegre,R,"21Â°51Â´16.556N""","102Â°22Â´21.884W""",21.854599,-102.372746,1879,F13D18,10,*,*,1
4,10010102,1,Aguascalientes,Ags.,1,Aguascalientes,102,Los Arbolitos [Rancho],R,"21Â°46Â´48.650N""","102Â°21Â´26.261W""",21.780181,-102.357295,1861,F13D18,7,*,*,1


In [43]:
coord_ags_1 = coordenadas[coordenadas.Cve_Ent ==1]
coord_bc = coordenadas[coordenadas.Cve_Ent ==2]

# MCP Catalog

In [5]:
municipios = pd.read_csv('C:/Users/Data Analyst/Documents/Otros/COVID-19_MX/coordenadas/catalogo_municipios.csv')
municipios.head()

,CLAVE_MUNICIPIO,MUNICIPIO,CLAVE_ENTIDAD
0,1,AGUASCALIENTES,1
1,2,ASIENTOS,1
2,3,CALVILLO,1
3,4,COSÍO,1
4,5,JESÚS MARÍA,1


In [41]:
# Aguascalientes
ags_mcp = municipios[municipios.CLAVE_ENTIDAD ==1]

# MX Covid Data

In [10]:
covid = pd.read_csv('C:/Users/Data Analyst/Documents/Otros/COVID-19_MX/covid_mx_12_05_20_ac1.csv')
covid.head(2)

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,DEFUNCION
0,12/05/2020,0ae84c,2,1,9,1,9,9,16,1,...,2,2,2,99,1,99,MÃ©xico,99,97,0
1,12/05/2020,1.14E+15,2,3,2,2,2,2,2,1,...,2,2,2,1,1,99,MÃ©xico,99,97,1


In [35]:
covid_ags = covid[covid.ENTIDAD_RES==1]
covid_ags.head(2)

C:\Users\Data Analyst\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,DEFUNCION,MUNICIPIO
128,12/05/2020,1b522e,2,4,1,1,1,1,1,1,...,2,2,99,1,99,MÃ©xico,99,97,1,AGUASCALIENTES
425,12/05/2020,16e77b,2,4,1,2,20,1,5,1,...,2,2,99,1,99,MÃ©xico,99,97,1,JESÚS MARÍA


# Mergin INEGI's coordenates with MCP catalog

In [42]:
# Aguascalientes
coord_mun_ags = pd.merge(ags_mcp, coord_ags_1[['Cve_Mun','Lat_Decimal','Lon_Decimal']],left_on = 'CLAVE_MUNICIPIO', right_on = 'Cve_Mun', how = 'inner' )
coord_mun_ags_clean = coord_mun_ags.groupby(['CLAVE_MUNICIPIO','MUNICIPIO'], as_index = False).mean()
coord_mun_ags_clean

,CLAVE_MUNICIPIO,MUNICIPIO,CLAVE_ENTIDAD,Cve_Mun,Lat_Decimal,Lon_Decimal
0,1,AGUASCALIENTES,1,1,21.831189,-102.282474
1,2,ASIENTOS,1,2,22.140375,-102.041058
2,3,CALVILLO,1,3,21.880951,-102.715365
3,4,COSÍO,1,4,22.358699,-102.280826
4,5,JESÚS MARÍA,1,5,21.947068,-102.386755
5,6,PABELLÓN DE ARTEAGA,1,6,22.118528,-102.282319
6,7,RINCÓN DE ROMOS,1,7,22.216246,-102.300355
7,8,SAN JOSÉ DE GRACIA,1,8,22.160838,-102.530186
8,9,TEPEZALÁ,1,9,22.226426,-102.231797
9,10,EL LLANO,1,10,21.881242,-102.077864
